In [1]:
import pandas as pd


In [2]:
applications_df = pd.read_csv("../data/application_record.csv")
credits_df = pd.read_csv("../data/credit_record.csv")


In [3]:
applications_df = applications_df.drop_duplicates(
    subset=applications_df.columns.difference(["ID"])
)


In [4]:
applications_df.head()


,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
2,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0
3,5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0
7,5008812,F,N,Y,0,283500.0,Pensioner,Higher education,Separated,House / apartment,-22464,365243,1,0,0,0,NaN,1.0
10,5008815,M,Y,Y,0,270000.0,Working,Higher education,Married,House / apartment,-16872,-769,1,1,1,1,Accountants,2.0


In [5]:
credits_df.head()


,ID,MONTHS_BALANCE,STATUS
0,5001711,0,X
1,5001711,-1,0
2,5001711,-2,0
3,5001711,-3,0
4,5001712,0,C


In [6]:
credits_df = credits_df[credits_df["MONTHS_BALANCE"] > -5]

In [7]:
applications_df = applications_df[applications_df["ID"].isin(credits_df["ID"])]
credits_df = credits_df[credits_df["ID"].isin(applications_df["ID"])]

In [8]:
def determine_approval(df):
    if df["STATUS"] == "C" or df["STATUS"] == "X":
        return 1
    else:
        return 0

In [9]:
credits_df["APPROVED"] = credits_df.apply(determine_approval, axis=1)

In [10]:
credits_df = (
    credits_df.groupby("ID")
    .agg(mean_approved=("APPROVED", "mean"))
    .reset_index()
)

In [11]:
credits_df["APPROVED"] = (credits_df["mean_approved"] > 0.5).astype(int)

In [12]:
credits_df = credits_df.drop("mean_approved", axis=1)

In [13]:
credits_df


,ID,APPROVED
0,5008804,1
1,5008806,1
2,5008808,1
3,5008812,0
4,5008815,0
...,...,...
7171,5150337,0
7172,5150388,1
7173,5150451,0
7174,5150459,1


In [14]:
merged_df = pd.merge(applications_df, credits_df, on="ID")

In [15]:
merged_df = merged_df[abs(merged_df["DAYS_EMPLOYED"]) <= 14600]

In [16]:
merged_df


,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,APPROVED
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0,1
1,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0,1
2,5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0,1
4,5008815,M,Y,Y,0,270000.0,Working,Higher education,Married,House / apartment,-16872,-769,1,1,1,1,Accountants,2.0,0
5,5008825,F,Y,N,0,130500.0,Working,Incomplete higher,Married,House / apartment,-10669,-1103,1,0,0,0,Accountants,2.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7171,5148694,F,N,N,0,180000.0,Pensioner,Secondary / secondary special,Civil marriage,Municipal apartment,-20600,-198,1,0,0,0,Laborers,2.0,0
7172,5149055,F,N,Y,0,112500.0,Commercial associate,Secondary / secondary special,Married,House / apartment,-15837,-2694,1,1,1,0,NaN,2.0,1
7173,5149729,M,Y,Y,0,90000.0,Working,Secondary / secondary special,Married,House / apartment,-19101,-1721,1,0,0,0,NaN,2.0,0
7174,5149838,F,N,Y,0,157500.0,Pensioner,Higher education,Married,House / apartment,-12387,-1325,1,0,1,1,Medicine staff,2.0,1


In [17]:
merged_df["OCCUPATION_TYPE"].fillna("Other", inplace=True)

In [18]:
merged_df.isna().sum()


ID                     0
CODE_GENDER            0
FLAG_OWN_CAR           0
FLAG_OWN_REALTY        0
CNT_CHILDREN           0
AMT_INCOME_TOTAL       0
NAME_INCOME_TYPE       0
NAME_EDUCATION_TYPE    0
NAME_FAMILY_STATUS     0
NAME_HOUSING_TYPE      0
DAYS_BIRTH             0
DAYS_EMPLOYED          0
FLAG_MOBIL             0
FLAG_WORK_PHONE        0
FLAG_PHONE             0
FLAG_EMAIL             0
OCCUPATION_TYPE        0
CNT_FAM_MEMBERS        0
APPROVED               0
dtype: int64

In [19]:
merged_df["APPROVED"].value_counts()

APPROVED
1    3964
0    1959
Name: count, dtype: int64

In [21]:
merged_df.to_csv("../data/merged_data.csv", index=False)
